In [1]:
import autograd.numpy as np
from autograd import grad

In [2]:
class NANDNeuron():
    def __init__(self, n):
        self.n = n
        self.epsilons = np.full((n), 0.0) #np.random.uniform(-1.0, 1.001, n)#
        
    def __repr__(self):
        return str(self.epsilons)
        
    def __beforePresent__(self):
        self.delta = 0
    
    def getDelta(self):
        return self.delta
    
    def setDelta(self, delta):
        self.delta = delta
    
    def present(self, inputs):
        self.__beforePresent__()
        # Compute a "logical" or on inputs and weights
        mus = (self.epsilons + inputs) - np.multiply(self.epsilons, inputs)

        self.mus = mus
        self.inputs = inputs
        
        out = np.prod(mus)
        self.output = 1 - out
        return self.output

    def getInput(self):
        return self.inputs
    
    def getMus(self):
        return self.mus
    
    def getOutput(self):
        return self.output
    
    def updateWeights(self, grad):
        self.epsilons = self.epsilons - grad
    
    def getWeights(self):
        return self.epsilons
    
    def getEffector(self):
        return self.effector

class NANDLayer():
    def __init__(self, inputs, nodes):
        self.layer = []
        
        for i in range(0, nodes):
            self.layer.append(NANDNeuron(inputs))
            
    def __repr__(self):
        s = ""
        for l in self.layer:
            s += (str(l) + " ,")
            
        return s

    def getLayer(self):
        return self.layer
    
    def setFolowingLayer(self, l):
        self.folowingLayer = l
        
    def present(self, inputs):
        out = []
        for n in self.layer:
            out.append(n.present(inputs))
        
        return out
    
    def backprop(self, prediction, target, output=False):
        for n in range(0, len(self.layer)):
            grad = None
            if output:
                grad = gradientOutputLayer(self.layer[n], target, prediction)
            else:
                grad = gradientHiddenLayer(self.layer[n], n, self.folowingLayer)
                
            self.layer[n].updateWeights(grad * 0.05)
    
class NANDNetwork():
    def __init__(self, nIns, lParams, nOuts):
        self.layers = []
        
        lParams.append(nOuts)
        inputs = nIns
        for l in lParams:
            self.layers.append(NANDLayer(inputs, l))
            inputs = l
                    
        for i in range(1, len(self.layers)):
            self.layers[i-1].setFolowingLayer(self.layers[i])
        
    def __repr__(self):
        s = ""
        for l in range(0, len(self.layers)):
            s += ("Layer " + str(l+1) + " -> " + str(self.layers[l]) + "\n")
            
        return s
        
    def fowardprop(self, inputs):
        for l in self.layers:
            inputs = l.present(inputs)
            
        return inputs[0]
            
    def backprop(self, prediction, target):
        for i in range(len(self.layers)-1, -1, -1):
            layer = self.layers[i]
            layer.backprop(prediction, target, i==(len(self.layers) - 1))
    
    
def MSE(network, data, targets):
    expected = np.array(list(map(lambda x: network.fowardprop(x), data)))
#     print(expected)
    return (1.0/2.0) * np.sum(np.power(np.subtract(expected, targets), 2))

def gradientHiddenLayer(neuron, neuronNumber, folowingLayer):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    # Compute the delta of current node
    dE_dy = 0
    for i in range(0, len(folowingLayer.getLayer())):
        fn = folowingLayer.getLayer()[i]
        dE_dyi = fn.getDelta()
        dxi_dyj = (1-fn.getWeights()[neuronNumber]) * np.prod(np.delete(fn.getMus(), neuronNumber))
        
        dE_dy -= dE_dyi * dxi_dyj
        
    neuron.setDelta(dE_dy)
    
    for i in range(0, numWeights):
        dx_dw = (1 - neuron.getWeights()[i]) * np.prod(np.delete(neuron.getMus(), i))
        
        grad[i] = -dE_dy * dx_dw
    
    return grad
    
    
def gradientOutputLayer(neuron, target, prediction):
    numWeights = len(neuron.getWeights())
    grad = np.zeros(numWeights)
    
    dE_dy = -(target - prediction)
        
    for i in range(0, numWeights):
        dx_dw = (1 - neuron.getWeights()[i]) * np.prod(np.delete(neuron.getMus(), i))
        
        grad[i] = -dE_dy * dx_dw

    neuron.setDelta(dE_dy)
    return grad


In [7]:
def trainNANDNetwork(data, targets, inputNodes, hLayers, outNodes):
    network = NANDNetwork(inputNodes, hLayers, outNodes)
    print(network)
    print("Initial Loss: ", MSE(network, data, targets))
    
    for i in range(1, 100000):   
        if (MSE(network, data, targets) < 0.0000000000001):
            break
            
        for j in range(0, len(data)):
            prediction = network.fowardprop(data[j])
            network.backprop(prediction, targets[j])
            
    print("Trained Loss: ", MSE(network, data, targets))
    return network
            
            

In [8]:
# NAND Gate Data
# data = np.array([[0.0,0.0],[1.0,0.0],[0.0,1.0],[1.0,1.0]])
# targets = np.array([1.0,1.0,1.0,0.0])



# NOT Gate Data
# data = np.array([[0.0, 1.0], [1.0, 1.0]])
# targets = np.array([1.0, 0.0])
# trainNANDNetwork(data, targets, 2, [], 1)


# AND Gate Data
# data = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
# targets = np.array([0.0, 0.0, 0.0, 1.0])

# OR Gate Data 
data = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targets = np.array([0.0, 1.0, 1.0, 1.0])

trainNANDNetwork(data, targets, 2, [2], 1)


Layer 1 -> [ 0.  0.] ,[ 0.  0.] ,
Layer 2 -> [ 0.  0.] ,

Initial Loss:  1.0
Trained Loss:  0.375044117233


Layer 1 -> [ 1.  1.] ,[ 1.  1.] ,
Layer 2 -> [ 0.49528107  0.49528107] ,